# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [6]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, Developer, and QA Engineer. Each role has specific responsibilities and skills tailored to their expertise within the collaborative framework, working together in a sequential workflow following Standard Operating Procedures (SOPs) to achieve efficient problem-solving and software development processes.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, a publish-subscribe mechanism, and a shared message pool. They utilize role-specific interests to 

In [7]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-30
last_modified_date: 2024-06-24

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [8]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and SoftwareDev. HumanEval consists of handwritten programming tasks, MBPP comprises Python tasks, and SoftwareDev contains diverse software development tasks such as mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the performance of MetaGPT in code generation and software development tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and SoftwareDev. HumanEval consists of handwritten programming tasks, MBPP comprises Python tasks, and SoftwareDev contains diverse software development tasks such as mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the

In [9]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["6"]}
=== Function Output ===
The results over the HumanEval dataset were utilized in the experimental setting.
=== LLM Response ===
The results over the HumanEval dataset were utilized in the experimental setting.


## Lower-Level: Debuggability and Control

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [12]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, Developer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and generating a Product Requirement Document (PRD). The Architect translates requirements into system design components, while the Project Manager handles task distribution. Engineers and Developers work on implementing the software based on specifications, and the QA Engineer formulates test cases to ensure code quality. These roles collaborate in a sequential manner following Standard Operating Procedures (SOPs) to efficiently tackle complex software development tasks in MetaGPT.


In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 07977456-e784-431c-abb6-23208d761c61: 1
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, Developer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and generating a Product Requirement Document (PRD). The Architect translates requirements into system design components, while the Project Manager handles task distribution. Engineers and Developers work on implementing the software based on specifications, and the QA Engineer formulates test cases to ensure code quality. These roles collaborate in a sequential manner following Standard Operating Procedures (SOPs) to efficiently tackle complex software development tasks in MetaGPT.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 07977456-e784-431c-abb6-23208d761c61: 1


TaskStep(task_id='07977456-e784-431c-abb6-23208d761c61', step_id='9ccaf8d6-699d-46ee-914e-1f850ccc0754', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [15]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. Additionally, agents can obtain directional information from other roles and public information from the environment. This structured communication interface enhances role communication efficiency within the framework. The shared message pool enables agents to exchange messages directly, publish their structured messages, access messages from other entities transparently, and utilize a subscription mechanism based on their role-specific interests to extract relevant information. This efficient sharing mechanism facilitates the sharing of i

In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. Additionally, agents can obtain directional information from other roles and public information from the environment. This structured communication interface enhances role communication efficiency within the framework. The shared message pool enables agents to exchange messages directly, publish their structured messages, access messages from other entities transparently, and utilize a subscription mechanism based on their role-specific interests to extract relevant information. This efficient sharing mechanism facilitates the sharing of information among agents within the simulated software company framework.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. Additionally, agents can obtain directional information from other roles and public information from the environment. This structured communication interface enhances role communication efficiency within the framework. The shared message pool enables agents to exchange messages directly, publish their structured messages, access messages from other entities transparently, and utilize a subscription mechanism based on their role-specific interests to extract relevant information. This efficient sharing mechanism facilitates the sharing of information among agents within the simulated software company framework.
